In [15]:
import pytds
import re 
import spacy
import pandas as pd
from time import time
connect_string = ""
try:
    conn = pytds.connect(server = '2605:ad80:30:5058:84ea:3de3:39a1:f11', database = 'TBDraft', user = 'admin1', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is not working


In [7]:
from tika import parser
import os
df = pd.DataFrame(columns=['CandidateID','Keyword', 'Resume', 'Cleaned_Resume'])
for dirname , _, filenames in os.walk('C:/Users/maxbr/ECEN 403/ECEN-403/Test Resumes'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4],'Resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'Cleaned_Resume' : ""}, 
                ignore_index = True)
   
df.shape

def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
df['Cleaned_Resume'] = df.Resume.apply(lambda x: cleanResume(x))
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Cleaned_Resume'])

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
df['Cleaned_Resume'] = txt
display(df.head())

Time to clean up everything: 0.01 mins


,CandidateID,Keyword,Resume,Cleaned_Resume
0,Andrew-Esfandiari,NaN,"Andrew Esfandiari Federal Way, WA -Email me ...",andrew esfandiari federal way wa email autho...
1,Arturo-Cervantes,NaN,"Arturo Cervantes Mission, TX -Email me on In...",arturo cervante mission tx email seek employ...
2,Bobby-Crenshaw,NaN,Bobby Crenshaw Lead ad-hoc trials aimed - Wi...,bobby crenshaw lead ad hoc trial aim windstr...
3,Brandon-Karnoski,NaN,"Brandon Karnoski League City, TX -Email me o...",brandon karnoski league city tx email work e...
4,Damien-Jeppson,NaN,"Damien Jeppson College Station, TX -Email me...",damien jeppson college station tx email seni...


In [10]:
df_students = pd.read_sql_query("SELECT ALL * FROM StudentInformation", conn)
display(df_students)


,ID,StudentName,GPA,Email,StudentResume,ProcessedData,PhoneNumber
0,10,None,None,None,b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,None,None
1,11,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
2,12,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
3,13,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
4,14,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None


In [12]:
cursor = conn.cursor()
cursor.execute('''TRUNCATE TABLE StudentInformation''')
conn.commit()

In [13]:
df_students = pd.read_sql_query("SELECT ALL * FROM StudentInformation", conn)
display(df_students)

,ID,StudentName,GPA,Email,StudentResume,ProcessedData,PhoneNumber


In [14]:
#cursor.execute("INSERT INTO HumanResources.DepartmentTest (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)
insert_q = "INSERT INTO StudentInformation (ID, StudentName, GPA, Email, StudentResume, ProcessedData, PhoneNumber) "
for i in range(len(df['Cleaned_Resume'])):

    values = "Values('', " + df.CandidateID[i] + ", '', '', " + df.Resume + ", '" + df.Cleaned_Resume + ", ''"")"
    commd = insert_q + values
    
    cursor.execute(commd)
    
conn.commit()

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", conn)
conn.close()
df_training

,keyword,dataText
0,Electronics_Engineer,andre salazar fort worth tx email talent ele...
1,Electronics_Engineer,arturo cervante mission tx email seek employ...
2,Electronics_Engineer,juan avila charleston sc email experience el...
3,Electronics_Engineer,travis ray engineer product designer develop...
4,Embedded_Systems,daniel koh forney tx email software developm...
...,...,...
101,Software_Engineer,kai schmidt software engineer citrus height ...
102,Software_Engineer,nathan nguyen irvine email work experience s...
103,Software_Engineer,peter pagano bohemia ny email authorize work...
104,Software_Engineer,steven nichol c c software engineer grand ra...


In [8]:
import pytds
connect_string = ""
try:
    conn = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is not working


In [31]:
df_roles = pd.read_sql_query("SELECT ALL * FROM Roles", conn)
df_projects = pd.read_sql_query("SELECT ALL * FROM Project", conn)

display(df_projects)
display(df_roles)

,projectName,Descrp,ProfessorName


,projectName,roleName,roleDescription,conskills,proskills,Candidate1,Candidate2,Candidate3
0,Solar Skateboard,Role 1,Power Supply Design,,"Power Supply, DC Converter",0,0,0
1,Solar Skateboard,Role 2,Microcontroller,,"Python, C, Embedded Systems",0,0,0
2,Solar Skateboard,Role 3,Microcontroller,Java,"Wireless Controller, RF",0,0,0
3,Music Project,Role 1,App Developer,,"Python, Coding, Java",0,0,0
4,Music Project,Role 2,Microcontroller/Bluetooth,Web Development,"Microcontroller, Wireless, Bluetooth",0,0,0
5,Music Project,Role 3,Audio Transmission,Power Supply Design,"ADC, Audio Transmission",0,0,0
6,Music Project,Role 4,Power Supply Design,,"Power Supply, Converter, Transformer",0,0,0


In [30]:
cursor = conn.cursor()
insert_q = "INSERT INTO Roles (projectName, roleName, roleDescription, Candidate1, Candidate2, Candidate3, proSkills, conSkills ) "
values = "Values('Music Project', 'Role 4', 'Power Supply Design', '' , '' , '', 'Power Supply, Converter, Transformer', '')"
commd = insert_q + values   
cursor.execute(commd)    
conn.commit()

In [14]:
# cursor.execute('''TRUNCATE TABLE Roles''')
# conn.commit()

In [ ]:
conn.close()